# Use Pre-Trained Model

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import torch
import tomli
import matplotlib.pyplot as plt
from os import environ
from pathlib import Path
from buildings_bench import load_torch_dataset
from buildings_bench.models import model_factory

/global/homes/n/nrushad/.conda/envs/BuildingsBenchEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DataHandler:
    def __init__(self): pass

    def load_dataset(self, dataset_name, scaler_transform):
        dataset = load_torch_dataset(
            name=dataset_name,
            apply_scaler_transform=scaler_transform,
            scaler_transform_path=Path(environ["TRANSFORM_PATH"])
        )
        return dataset

    def create_dataloader(self, building_dataset):
        dataloader = torch.utils.data.DataLoader(
                        building_dataset,
                        batch_size=360,
                        shuffle=False)
        return dataloader

In [7]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_dim=168, hidden_dim=1, layer_dim=1, output_dim=24):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out, hn, cn

In [12]:
def main(model_name, dataset_name, scaler_transform, device):
    handler = DataHandler()
    dataset = handler.load_dataset(dataset_name, scaler_transform)    

    model = LSTMModel(input_dim=1, hidden_dim=100, layer_dim=1, output_dim=1)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    target_list = []
    prediction_list = []

    num_epochs = 100
    h0, c0 = None, None


    for epoch in range(10):

        model.train()
        optimizer.zero_grad()
    
        for building_id, building_dataset in dataset:
            
            inverse_transform = building_dataset.datasets[0].load_transform.undo_transform
            dataloader = handler.create_dataloader(building_dataset)
            
            for batch in dataloader:
                for key, value in batch.items():
                    batch[key] = value.to(device)
                    print(batch)
                    break
    
                # with torch.cuda.amp.autocast(enabled=(device == 'cuda:0')):
                #     outputs, h0, c0 = model(batch, h0, c0) 
                #     print(outputs)
        


In [ ]:
if __name__ == "__main__":

    DATASET = 'ideal' # Electricity | SMART | Ideal | lcl
    DEVICE = 'cuda:0'
    MODEL = 'TransformerWithGaussian-L' # TransformerWithGaussian-S | TransformerWithGaussian-M
    CHECKPOINT = 'Transformer_Gaussian_L.pt'
    SCALER_TRANSFORM = 'boxcox'

    PATH = '/pscratch/sd/n/nrushad'
    environ["PATH"] = PATH
    
    environ["REPO_PATH"] = f'{PATH}/BuildingsBenchTutorial/BuildingsBench/'
    environ["BUILDINGS_BENCH"] = f'{PATH}/Dataset'
    environ["TRANSFORM_PATH"] = f'{PATH}/Dataset/metadata/transforms'
    
    main(MODEL, DATASET, SCALER_TRANSFORM, DEVICE)

{'latitude': tensor([[[3.2296],
         [3.2296],
         [3.2296],
         ...,
         [3.2296],
         [3.2296],
         [3.2296]],

        [[3.2296],
         [3.2296],
         [3.2296],
         ...,
         [3.2296],
         [3.2296],
         [3.2296]],

        [[3.2296],
         [3.2296],
         [3.2296],
         ...,
         [3.2296],
         [3.2296],
         [3.2296]],

        ...,

        [[3.2296],
         [3.2296],
         [3.2296],
         ...,
         [3.2296],
         [3.2296],
         [3.2296]],

        [[3.2296],
         [3.2296],
         [3.2296],
         ...,
         [3.2296],
         [3.2296],
         [3.2296]],

        [[3.2296],
         [3.2296],
         [3.2296],
         ...,
         [3.2296],
         [3.2296],
         [3.2296]]], device='cuda:0'), 'longitude': tensor([[[-5.2057],
         [-5.2057],
         [-5.2057],
         ...,
         [-5.2057],
         [-5.2057],
         [-5.2057]],

        [[-5.2057],
      